# Installing libraries

In [ ]:
!pip install openbiolink

     |████████████████████████████████| 245kB 22.7MB/s 
     |████████████████████████████████| 471kB 49.5MB/s 
     |████████████████████████████████| 286kB 50.1MB/s 
     |████████████████████████████████| 28.5MB 94kB/s 
     |████████████████████████████████| 307kB 53.9MB/s 
     |████████████████████████████████| 51kB 8.2MB/s 
     |████████████████████████████████| 174kB 56.8MB/s 
     |████████████████████████████████| 81kB 11.5MB/s 
     |████████████████████████████████| 81kB 12.3MB/s 
     |████████████████████████████████| 143kB 53.0MB/s 
     |████████████████████████████████| 112kB 46.8MB/s 
     |████████████████████████████████| 51kB 8.0MB/s 
  Created wheel for click-default-group: filename=click_default_group-1.2.2-cp37-none-any.whl size=3386 sha256=9c953480c40a37b1e549f475824ce98599f7fdd87367923e54151a2d995f874e
  Stored in directory: /root/.cache/pip/wheels/ca/ed/79/2200269c285b0d21ab4591c6388a14691a4fc77d76bec40fe2
  Created wheel for stringcase: filename=stringcase-

In [ ]:
%%capture 
!pip install ampligraph;

In [ ]:
%tensorflow_version 1.x 
import numpy as np
import pandas as pd
import ampligraph

ampligraph.__version__

TensorFlow 1.x selected.


'1.4.0'

# Loading OpenBioLink dataset

In [ ]:
from openbiolink.evaluation.dataLoader import DataLoader

# Name of the Dataset, possible values HQ_DIR, HQ_UNDIR, ALL_DIR, ALL_UNDIR. Default: HQ_DIR
dl = DataLoader("HQ_DIR")

HQ_DIR.zip: 0.00B [00:00, ?B/s]

Dataset not found in, downloading to /content/datasets/HQ_DIR ...


HQ_DIR.zip: 104MB [00:14, 7.70MB/s]                           


Loading dataset ...
Done!


In [ ]:
cmp = []

for el in dl.mappings['nodes']['label2id']:
  if 'compound' in el.lower():
    cmp.append(el)

In [ ]:
len(cmp)

77709

In [ ]:
rels = set(dl.data['train_positive'][1])
len(rels)

28

In [ ]:
len(dl.mappings['nodes']['label2id'])

184732

In [ ]:
dl.mappings['relations']['label2id']

{'DIS_DRUG': 25,
 'DIS_PHENOTYPE': 4,
 'DRUG_ACTIVATION_GENE': 9,
 'DRUG_BINDACT_GENE': 14,
 'DRUG_BINDING_GENE': 22,
 'DRUG_BINDINH_GENE': 5,
 'DRUG_CATALYSIS_GENE': 18,
 'DRUG_INHIBITION_GENE': 21,
 'DRUG_PHENOTYPE': 27,
 'DRUG_REACTION_GENE': 20,
 'GENE_ACTIVATION_GENE': 23,
 'GENE_BINDING_GENE': 12,
 'GENE_CATALYSIS_GENE': 24,
 'GENE_DIS': 10,
 'GENE_DRUG': 2,
 'GENE_EXPRESSED_ANATOMY': 15,
 'GENE_EXPRESSION_GENE': 19,
 'GENE_GENE': 0,
 'GENE_GO': 8,
 'GENE_INHIBITION_GENE': 16,
 'GENE_OVEREXPRESSED_ANATOMY': 13,
 'GENE_PATHWAY': 1,
 'GENE_PHENOTYPE': 7,
 'GENE_PTMOD_GENE': 26,
 'GENE_REACTION_GENE': 6,
 'GENE_UNDEREXPRESSED_ANATOMY': 3,
 'IS_A': 17,
 'PART_OF': 11}

In [ ]:
train_positive = dl.data['train_positive'].to_numpy()
train_negative = dl.data['train_negative'].to_numpy()
test_positive = dl.data['test_positive'].to_numpy()
test_negative = dl.data['test_negative'].to_numpy()
validation_positive = dl.data['valid_positive'].to_numpy()
vakidation_negative = dl.data['valid_negative'].to_numpy()

# Model creation

In [ ]:
from ampligraph.latent_features import ComplEx

model = ComplEx(batches_count=250, 
                seed=0, 
                epochs=50, 
                k=200, 
                eta=30,
                embedding_model_params = {'negative_corruption_entities': train_negative},
                optimizer='adam', 
                optimizer_params={'lr':0.001},
                loss='multiclass_nll', 
                regularizer='LP', 
                regularizer_params={'p':3, 'lambda':0.001}, 
                verbose=True)

# Model training

In [ ]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
model.fit(train_positive, early_stopping = False)

/usr/local/lib/python3.7/dist-packages/ampligraph/latent_features/models/EmbeddingModel.py:652: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if negative_corruption_entities == 'all':
/usr/local/lib/python3.7/dist-packages/ampligraph/latent_features/models/EmbeddingModel.py:663: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif negative_corruption_entities == 'batch':
Average ComplEx Loss:   0.840990: 100%|██████████| 50/50 [1:12:33<00:00, 87.07s/epoch]


In [ ]:
from ampligraph.latent_features import save_model, restore_model

save_model(model, './complex_model_OpenBioLink.pkl')

# Model evaluation

In [ ]:
X_positive = np.concatenate((train_positive, test_positive, validation_positive), axis=0)
len(X_positive)

4563405

In [ ]:
from ampligraph.evaluation import evaluate_performance

ranks = evaluate_performance(test_positive, 
                             model=model, 
                             filter_triples=X_positive,   # Corruption strategy filter defined above 
                             verbose=True)

WARNING - You are attempting to use 180992 distinct entities to generate synthetic negatives in the evaluation
    protocol. This may be unnecessary and will lead to a 'harder' task. Besides, it will lead to a much slower
    evaluation procedure. We recommended to set the 'corruption_entities' argument to a reasonably sized set
    of entities. The size of corruption_entities depends on your domain-specific task.


/usr/local/lib/python3.7/dist-packages/ampligraph/evaluation/protocol.py:952: UserWarning: You are attempting to use 180992 distinct entities to generate synthetic negatives in the evaluation
    protocol. This may be unnecessary and will lead to a 'harder' task. Besides, it will lead to a much slower
    evaluation procedure. We recommended to set the 'corruption_entities' argument to a reasonably sized set
    of entities. The size of corruption_entities depends on your domain-specific task.
  warnings.warn(warn_msg % ent_for_corruption_size)
 62%|██████▏   | 111593/180964 [2:37:36<1:38:09, 11.78it/s]

In [ ]:
from numpy import asarray
from numpy import savetxt

data = asarray(ranks)
# save to csv file
savetxt('ranks_complex.csv', data, delimiter=',')

In [ ]:
from ampligraph.evaluation import mr_score, mrr_score, hits_at_n_score

mrr = mrr_score(ranks)
print("MRR: %.2f" % (mrr))

hits_10 = hits_at_n_score(ranks, n=10)
print("Hits@10: %.2f" % (hits_10))
hits_3 = hits_at_n_score(ranks, n=3)
print("Hits@3: %.2f" % (hits_3))
hits_1 = hits_at_n_score(ranks, n=1)
print("Hits@1: %.2f" % (hits_1))